In [79]:
import numpy as np
import pandas as pd
import nltk

colnames = ['text','label']
#df = pd.read_csv("https://media-doselect.s3.amazonaws.com/generic/O0nEJwARYWpLWP40PQwdYa023/training-book_reviews.zip")
df = pd.read_csv("book_reviews.csv", names = colnames, header = None)

count_rows = len(df)
#with open("/code/output/output1.csv", mode = "w") as f:
with open("output1.csv", mode = "w") as f:
    f.write(str(count_rows))
    
from nltk.tokenize import TweetTokenizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

def tokenize(strings): 
    tk = TweetTokenizer()
    return tk.tokenize(strings)

vectorizer = CountVectorizer(analyzer = 'word', tokenizer = tokenize, lowercase = True, ngram_range = (1, 1))

unique_words = vectorizer.fit_transform(df['text'])
rows, columns = unique_words.shape

with open("output2.csv", mode = "w") as f:
    f.write(str(columns))
    
X = df['text'].values
y = df['label'].values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 100, test_size = 0.3)

vectorizer = TfidfVectorizer(max_features = 1000)
X_train_idf = vectorizer.fit_transform(X_train)
X_test_idf = vectorizer.transform(X_test)

df_idf = pd.DataFrame(vectorizer.idf_, index = vectorizer.get_feature_names(), columns = ["idf_weights"])
df_idf_sorted = df_idf.sort_values(by = ['idf_weights'], ascending = False)
top_5_idf = df_idf_sorted['idf_weights'][0:5].tolist()
top_5_idf_r = [round(num, 3) for num in top_5_idf]

with open("output3.csv", mode = "w") as f:
    for num in top_5_idf_r:
        f.write(str(num))
        f.write("\n")
        
from nltk import word_tokenize, pos_tag 
def clean_text(text):
    count = sum(1 for word, pos in pos_tag(word_tokenize(text)) if pos.startswith('NN'))
    return (count)

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, accuracy_score

mnb = MultinomialNB()
mnb.fit(X_train_idf, y_train)

pred_mnb = mnb.predict(X_test_idf)
acc = round(accuracy_score(y_test, pred_mnb), 3)

with open("output4.csv", mode = "w") as f:
    f.write(str(acc))

pd.DataFrame(confusion_matrix(y_test, pred_mnb)).to_csv("output5.csv")